# Video Parsing Pipeline

Required: 
- csv containing video links

Functions:
- downloads linked videos
- separates into 
    - audio (.wav file)
    - speech transcript (if applicable)
    - individual frames (.jpg, sampled at 1 per second)

In [1]:
# global variables (change to your preference)
LINK_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links"
PARENT_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_vids"
CSV_DIR = "mk_superbowl_ads_mar2023_v3.csv"
VIDLINK_COL = "ad_vid_link"

In [2]:
# imports
import os
import pandas as pd
import urllib
import time
import random
import os
import requests
import cv2

In [3]:
# cd into parent directory, read contents of csv

os.chdir(LINK_DIR)
df = pd.read_csv(CSV_DIR)

## Downloading Videos

In [4]:
# read and download all videos
def download_videos(ID):
    
    os.chdir(PARENT_DIR)

    file_name = ID +".mp4"
    link = df[VIDLINK_COL][i]

    # make seperate directory
    os.mkdir(f'{PARENT_DIR}/{ID}')

    print("Downloading file:%s"%file_name)

    # create response object 
    r = requests.get(link, stream = True) 

    # download started 
    with open(f'{PARENT_DIR}/{ID}/{file_name}', 'wb') as f: 
        for chunk in r.iter_content(chunk_size = 1024*1024): 
            if chunk: 
                f.write(chunk) 

    print("%s downloaded!\n"%file_name) 

    time.sleep(random.randint(0, 3))


    print("All videos downloaded!")

## Scene Detection

In [5]:
import subprocess
from scenedetect import detect, ContentDetector, AdaptiveDetector


def find_scenes(filename):
    scene_list = detect(filename, AdaptiveDetector(adaptive_threshold=3.0, min_scene_len=48))
    return len(scene_list)

In [6]:
import datetime

def find_length(filename):
    data = cv2.VideoCapture(filename)
    if not data.isOpened():
        df2['has_vid'][i] = 0
        return 0
    
    # count the number of frames
    frames = data.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = data.get(cv2.CAP_PROP_FPS)
    if frames == 0 or fps == 0: 
        df2['has_vid'][i] = 0
        return 0

    # calculate duration of the video
    seconds = round(frames / fps, 2)
    video_time = datetime.timedelta(seconds=seconds)
    return seconds

In [7]:
old_filename = "nw_nov_v5
new_filename = "nw_april_v1.csv"
def scene_length_detect(df):
    print('hi')

    os.chdir(PARENT_DIR)
    df2 = df.copy()
    # df["num_scenes"] = [None for x in range(df.shape[0])]
    # df["length"] = [None for x in range(df.shape[0])]

    for i in range(df.shape[0]):
        
        if df['has_vid'][i] == 0:
            continue
        elif not pd.isnull(df['num_scenes'][i]):
            continue
        

        ID = df['id'][i]
        file_name = ID +".mp4"
        
        
        try:
            os.chdir(f'{PARENT_DIR}/{ID}')
            scenes = find_scenes(file_name)
            length = find_length(file_name)
            df2["num_scenes"][i] = scenes
            df2["length"][i] = length
            print(f'done with {ID}')
        except:
            pass
    
    print('Finished!')
    
    os.chdir(LINK_DIR)
    df2.to_csv(new_file_name)


In [9]:
os.chdir(LINK_DIR)

df = pd.read_csv(old_filename)
scene_length_detect(df)

In [9]:
os.chdir(LINK_DIR)
df = pd.read_csv("nw_temp_v3.csv")
scene = pd.read_csv("nw_superbowl_scene_feb2023_v3.csv")
combined_df = df.copy()
combined_df["num_scenes"] = scene["num_scenes"]
combined_df["length"] = scene["length"]
combined_df.to_csv("nw_temp_v4.csv")

## Extract Audio

In [29]:
import moviepy.editor as mp

def extract_audio(ID):
    # for i in range(df.shape[0]):
        # ID = df['id'][i]
    file_name = ID +".mp4"

    os.chdir(f'{PARENT_DIR}/{ID}')

    my_clip = mp.VideoFileClip(file_name)

    try:
        my_clip.audio.write_audiofile(f'{ID}_audio.wav')
    except:
        print(f'{ID} has no audio')   
    print("Finished extracting audio!")

## Extract Text from .wav file

Install google cloud speech if needed (pip install google-cloud-speech)

In [30]:
import speech_recognition as sr

def extract_text(ID):

# for i in range(df.shape[0]):
    
    # ID = df['id'][i]
    file_name = ID +".mp4"
    
    os.chdir(f'{PARENT_DIR}/{ID}')
    
    r = sr.Recognizer()
    wav = sr.AudioFile(f'{ID}_audio.wav')
    
    with wav as source:
        audio = r.record(source)
    try:
        # for testing purposes, we're just using the default API key
        print(f'trying to parse {ID}')
        text = r.recognize_google(audio)
        print("Google Speech Recognition thinks you said " + text)
        f = open(f"{ID}_transcript.txt", "w")
        f.write(text)
        f.close()
        
    except sr.UnknownValueError:
        print("no speech detected")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    
    print('finished')

## Saving Frames Individually

In [31]:
def clean_frame_mk(img, sz = 256.0):
    h = img.shape[0]
    w = img.shape[1]
    nw = w*0.8
    nh = h*0.8
    left = (w - nw)/2
    top = (h - nh)/2
    right = (w + nw)/2
    bottom = (h + nh)/2
    img = img[int(top):int(bottom), int(left):int(right)]
    r = img.shape[0]/sz
    dim = (int(img.shape[1]/r), int(sz))  # resize & maintain the aspect ratio
    # useful for extracting other features such as presence of objects and human faces
    # results are unchanged if we hold size constant
    processed_img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
    return processed_img

In [32]:
def save_frames(ID):
    file_name = ID +".mp4"

    # move into correct directory
    os.chdir(f'{PARENT_DIR}/{ID}')

    cap = cv2.VideoCapture(file_name)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # make seperate directory
    os.mkdir(f'{PARENT_DIR}/{ID}/img_frames')

    count, frame_num = 0, 1
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if count % fps == 0:

            if ret:
                frame_name = f'{ID}_frame_{frame_num}.jpg'
                cv2.imwrite(f'{PARENT_DIR}/{ID}/img_frames/{frame_name}', clean_frame_mk(frame))
                frame_num += 1
            else:
                break
        
        count += 1

    if not cap.isOpened():
        print('novideo')
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    print(ID+" done saving")

    print('finished')
    

## Run all functions

In [53]:
import shutil
# go through csv, for current purposes we only run on videos with "has_vid" = 0
os.chdir(LINK_DIR)

no_vid = []
df = pd.read_csv("nw_temp_v2.csv") 
df2 = df.copy()
df2 = df2.drop('has_vid', axis=1)
df2['has_vid'] = [None for x in range(df.shape[0])]


for i in range(3147, df.shape[0]):
        
    if df["has_vid"][i] == 1:
        continue
    
    ID = df["id"][i]
    file_name = ID + ".mp4"

    download_videos(ID)
        
    try:
        os.chdir(f'{PARENT_DIR}/{ID}')
        cap = cv2.VideoCapture(file_name)
        ret, frame = cap.read()
        extract_audio(ID)
        save_frames(ID)
    except:
        no_vid.append(ID)
        df2['has_vid'][i] = 0
        try:
            shutil.rmtree(f'{PARENT_DIR}/{ID}')
        except:
            pass
        continue
    
    extract_text(ID)

    
    df2['has_vid'][i] = 1
    print(f'finished processing {ID}')

print('DONE')
os.chdir(LINK_DIR)
df2.to_csv("nw_temp_v3.csv")

sb_2006_3099.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_2006_3099_audio.wav


MoviePy - Done.
Finished extracting audio!
sb_2006_3099 done saving
finished
trying to parse sb_2006_3099
result2:
{   'alternative': [   {   'confidence': 0.9581973,
                           'transcript': 'introducing the miracle of fusion a '
                                         'revolutionary technology come '
                                         'together to create'},
                       {   'transcript': 'introducing the miracle of fusion a '
                                         'revolutionary technology has a '
                                         'unique idea come together to create'},
                       {   'transcript': 'introducing the miracle of fusion a '
                                         'revolutionary technology and the '
                                         'unique idea come together to create'},
                       {   'transcript': 'introducing the miracle of fusion a '
                                         'revolutionary techn

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5555577608c0] moov atom not found


sb_2006_3108.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_2006_3108_audio.wav


MoviePy - Done.
Finished extracting audio!
sb_2006_3108 done saving
finished
trying to parse sb_2006_3108
result2:
{   'alternative': [   {'confidence': 0.97219545, 'transcript': 'period'},
                       {'transcript': 'period smiley face'},
                       {'transcript': 'period question mark'},
                       {'transcript': 'period smiley face question mark'},
                       {'transcript': 'period period'}],
    'final': True}
Google Speech Recognition thinks you said period
finished
finished processing sb_2006_3108
sb_2006_3115.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_2006_3115_audio.wav


...

MoviePy - Done.
Finished extracting audio!
sb_1986_3503 done saving
finished
trying to parse sb_1986_3503
result2:
[]
no speech detected
finished
finished processing sb_1986_3503
sb_1986_3504.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_1986_3504_audio.wav


MoviePy - Done.
Finished extracting audio!
sb_1986_3504 done saving
finished
trying to parse sb_1986_3504
result2:
[]
no speech detected
finished
finished processing sb_1986_3504
sb_1986_3506.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_1986_3506_audio.wav


MoviePy - Done.
Finished extracting audio!


sb_1986_3506 done saving
finished
trying to parse sb_1986_3506
result2:
[]
no speech detected
finished
finished processing sb_1986_3506
sb_1986_3507.mp4 downloaded!

All videos downloaded!
MoviePy - Writing audio in sb_1986_3507_audio.wav


MoviePy - Done.
Finished extracting audio!
sb_1986_3507 done saving
finished
trying to parse sb_1986_3507
result2:
{   'alternative': [   {   'confidence': 0.94926387,
                           'transcript': 'this is the home of the world famous '
                                         "Clydesdales it's a symbol of a long "
                                         'tradition of quality that goes into '
                                         'making Budweiser along with the '
                                         'finest ingredients and Beachwood '
                                         'aging Budweiser is brewed with a '
                                         'Special Touch of care and pride no '
                                         'other Brewer has such a traditional '
                                         'quality to live up to but then no '
                                         "other group after all this bud's for "
                                         'y

In [54]:
print('hello')

hello


In [41]:
# print(no_vid)
unique_no_vid = set(no_vid)
print(unique_no_vid)

{'sb_2006_3104', 'sb_2022_83', 'sb_1987_2596', 'sb_2004_1273', 'sb_2006_3130', 'sb_1999_1668', 'sb_1994_2022', 'sb_2006_1086', 'sb_1999_1654', 'sb_1992_2225', 'sb_1990_3447', 'sb_2004_1263', 'sb_2022_3072', 'sb_2005_1212', 'sb_2022_82', 'sb_2007_1047', 'sb_1980_3027', 'sb_1994_3407', 'sb_2005_1213', 'sb_1989_2463', 'sb_1988_2506', 'sb_2023_3509', 'sb_2004_1242', 'sb_2004_1226', 'sb_1994_2073', 'sb_1989_2421', 'sb_1985_2767', 'sb_2004_3208', 'sb_2022_3071', 'sb_1987_2572', 'sb_1987_2614', 'sb_1988_2539', 'sb_2021_120', 'sb_1999_1676', 'sb_2004_1280', 'sb_2004_3219', 'sb_1986_2640', 'sb_1987_2615', 'sb_1985_2746', 'sb_1994_2029', 'sb_1984_2792', 'sb_2004_3232', 'sb_2012_701', 'sb_1989_2401', 'sb_2022_3073', 'sb_2005_1214', 'sb_1995_1928', 'sb_2022_81'}


## Redownload frames for each video (1 per second)

In [16]:
no_vid2 = []
for i in range(df.shape[0]):
    ID = df["id"][i]
    
    try:
        save_frames(ID)
    except:
        no_vid2.append(ID)

print('finished saving frames!')
    

sb_2023_3509 done saving
finished
sb_2023_3510 done saving
finished
sb_2023_3511 done saving
finished
sb_2023_3512 done saving
finished
sb_2023_3513 done saving
finished
sb_2023_3514 done saving
finished
sb_2023_3515 done saving
finished
sb_2023_3516 done saving
finished
sb_2023_3517 done saving
finished
sb_2023_3518 done saving
finished
sb_2023_3519 done saving
finished
sb_2023_3520 done saving
finished
sb_2023_3521 done saving
finished
sb_2023_3522 done saving
finished
sb_2023_3523 done saving
finished
sb_2023_3524 done saving
finished
sb_2023_3525 done saving
finished
sb_2023_3526 done saving
finished
sb_2023_3527 done saving
finished
sb_2023_3528 done saving
finished
sb_2023_3532 done saving
finished
sb_2023_3533 done saving
finished
sb_2023_3534 done saving
finished
sb_2023_3535 done saving
finished
sb_2023_3536 done saving
finished
sb_2023_3537 done saving
finished
sb_2023_3538 done saving
finished
sb_2023_3539 done saving
finished
sb_2023_3540 done saving
finished
sb_2023_3541 d

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x555556385c40] moov atom not found


sb_2021_121 done saving
finished
sb_2021_122 done saving
finished
sb_2021_123 done saving
finished
sb_2021_124 done saving
finished
sb_2021_128 done saving
finished
sb_2021_129 done saving
finished
sb_2021_130 done saving
finished
sb_2021_132 done saving
finished
sb_2021_133 done saving
finished
sb_2021_134 done saving
finished
sb_2021_135 done saving
finished
sb_2021_136 done saving
finished
sb_2021_138 done saving
finished
sb_2021_139 done saving
finished
sb_2021_140 done saving
finished
sb_2021_142 done saving
finished
sb_2021_143 done saving
finished
sb_2021_144 done saving
finished
sb_2021_147 done saving
finished
sb_2021_148 done saving
finished
sb_2021_149 done saving
finished
sb_2021_151 done saving
finished
sb_2020_152 done saving
finished
sb_2020_153 done saving
finished
sb_2020_156 done saving
finished
sb_2020_158 done saving
finished
sb_2020_159 done saving
finished
sb_2020_160 done saving
finished
sb_2020_161 done saving
finished
sb_2020_162 done saving
finished
sb_2020_16

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x555556385c40] moov atom not found


sb_2012_702 done saving
finished
sb_2012_703 done saving
finished
sb_2012_704 done saving
finished
sb_2012_705 done saving
finished
sb_2012_706 done saving
finished
sb_2012_707 done saving
finished
sb_2012_708 done saving
finished
sb_2011_709 done saving
finished
sb_2011_710 done saving
finished
sb_2011_711 done saving
finished
sb_2011_712 done saving
finished
sb_2011_713 done saving
finished
sb_2011_714 done saving
finished
sb_2011_715 done saving
finished
sb_2011_716 done saving
finished
sb_2011_717 done saving
finished
sb_2011_718 done saving
finished
sb_2011_719 done saving
finished
sb_2011_720 done saving
finished
sb_2011_721 done saving
finished
sb_2011_722 done saving
finished
sb_2011_723 done saving
finished
sb_2011_724 done saving
finished
sb_2011_725 done saving
finished
sb_2011_726 done saving
finished
sb_2011_727 done saving
finished
sb_2011_728 done saving
finished
sb_2011_729 done saving
finished
sb_2011_730 done saving
finished
sb_2011_731 done saving
finished
sb_2011_73

...

sb_2003_3235 done saving
finished
sb_2003_3236 done saving
finished
sb_2003_3237 done saving
finished
sb_2003_3238 done saving
finished
sb_2003_3239 done saving
finished
sb_2003_3240 done saving
finished
sb_2003_3241 done saving
finished
sb_2003_3242 done saving
finished
sb_2003_3243 done saving
finished
sb_2003_3244 done saving
finished
sb_2003_3245 done saving
finished
sb_2003_3246 done saving
finished
sb_2003_3247 done saving
finished
sb_2003_3248 done saving
finished
sb_2003_3249 done saving
finished
sb_2003_3250 done saving
finished
sb_2003_3251 done saving
finished
sb_2003_3252 done saving
finished
sb_2003_3253 done saving
finished
sb_2003_3254 done saving
finished
sb_2003_3255 done saving
finished
sb_2003_3256 done saving
finished
sb_2003_3257 done saving
finished
sb_2003_3258 done saving
finished
sb_2003_3259 done saving
finished
sb_2003_3260 done saving
finished
sb_2003_3261 done saving
finished
sb_2003_3262 done saving
finished
sb_2003_3263 done saving
finished
sb_2003_3264 d

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x555556385c40] moov atom not found


sb_1993_3408 done saving
finished
sb_1993_3409 done saving
finished
sb_1993_3410 done saving
finished
sb_1993_3411 done saving
finished
sb_1993_3412 done saving
finished
sb_1991_3413 done saving
finished
sb_1991_3414 done saving
finished
sb_1991_3415 done saving
finished
sb_1991_3416 done saving
finished
sb_1991_3417 done saving
finished
sb_1991_3418 done saving
finished
sb_1991_3419 done saving
finished
sb_1991_3420 done saving
finished
sb_1991_3421 done saving
finished
sb_1991_3422 done saving
finished
sb_1991_3423 done saving
finished
sb_1991_3424 done saving
finished
sb_1991_3425 done saving
finished
sb_1991_3426 done saving
finished
sb_1991_3427 done saving
finished
sb_1991_3428 done saving
finished
sb_1991_3429 done saving
finished
sb_1991_3430 done saving
finished
sb_1991_3431 done saving
finished
sb_1991_3432 done saving
finished
sb_1991_3433 done saving
finished
sb_1991_3434 done saving
finished
sb_1991_3435 done saving
finished
sb_1991_3436 done saving
finished
sb_1991_3437 d

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x555556385c40] moov atom not found


sb_1990_3448 done saving
finished
sb_1990_3449 done saving
finished
sb_1990_3450 done saving
finished
sb_1990_3451 done saving
finished
sb_1987_3452 done saving
finished
sb_1987_3453 done saving
finished
sb_1987_3454 done saving
finished
sb_1987_3455 done saving
finished
sb_1987_3456 done saving
finished
sb_1987_3457 done saving
finished
sb_1987_3459 done saving
finished
sb_1987_3461 done saving
finished
sb_1987_3462 done saving
finished
sb_1987_3463 done saving
finished
sb_1987_3464 done saving
finished
sb_1987_3465 done saving
finished
sb_1987_3466 done saving
finished
sb_1987_3467 done saving
finished
sb_1987_3468 done saving
finished
sb_1987_3469 done saving
finished
sb_1987_3471 done saving
finished
sb_1987_3473 done saving
finished
sb_1987_3476 done saving
finished
sb_1986_3478 done saving
finished
sb_1986_3479 done saving
finished
sb_1986_3480 done saving
finished
sb_1986_3481 done saving
finished
sb_1986_3482 done saving
finished
sb_1986_3483 done saving
finished
sb_1986_3484 d